In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

# Define the namespace
namespace = {'ns': 'urn:cwa:gov:tw:cwacommon:0.1'}

# time scale
n_years = 5
years = [2022 - x for x in range(n_years)]
months = [f"{i:02d}" for i in range(1, 13)]


# concatenate all dfs
all_dfs = pd.DataFrame(columns=[
    'StationID',
    'StationName',
    'StationNameEN',
    'StationAttribute',
    'AirTemperatureMean',
    'AirTemperatureMax',
    'AirTemperatureMaxDate',
])


for year in years:
    print(f'\n===================================== {year} =====================================\n')

    for month in months:        
        
        # Parse the XML file
        tree = ET.parse(f'./data/{year}/mn_Report_{year}{month}.xml')
        root = tree.getroot()

        # Create lists to store data
        data = []

        # Iterate through each 'location' element
        for location in root.findall('.//ns:location', namespace):
            station_id = location.find('.//ns:StationID', namespace).text
            station_name = location.find('.//ns:StationName', namespace).text
            station_name_en = location.find('.//ns:StationNameEN', namespace).text
            station_attribute = location.find('.//ns:StationAttribute', namespace).text

            air_temperature_mean = location.find('.//ns:Mean', namespace).text
            air_temperature_max = location.find('.//ns:Maximum', namespace).text
            air_temperature_max_date = location.find('.//ns:MaximumDate', namespace).text

            # Append data to the list
            data.append({
                'StationID': station_id,
                'StationName': station_name,
                'StationNameEN': station_name_en,
                'StationAttribute': station_attribute,
                'AirTemperatureMean': air_temperature_mean,
                'AirTemperatureMax': air_temperature_max,
                'AirTemperatureMaxDate': air_temperature_max_date
            })

        # Create a DataFrame
        df = pd.DataFrame(data)
        
        all_dfs = pd.concat([all_dfs, df], ignore_index=True)

all_dfs


===================================== 2022 =====================================


===================================== 2021 =====================================


===================================== 2020 =====================================


===================================== 2019 =====================================


===================================== 2018 =====================================



,StationID,StationName,StationNameEN,StationAttribute,AirTemperatureMean,AirTemperatureMax,AirTemperatureMaxDate
0,466880,板橋,BANQIAO,署屬有人氣象站,17.2,26.4,2022-01-27
1,466900,淡水,TAMSUI,署屬有人氣象站,16.5,26.4,2022-01-26
2,466910,鞍部,ANBU,署屬有人氣象站,10.7,19.8,2022-01-27
3,466920,臺北,TAIPEI,署屬有人氣象站,17.2,25.8,2022-01-26
4,466930,竹子湖,ZHUZIHU,署屬有人氣象站,12.7,21.4,2022-01-26
...,...,...,...,...,...,...,...
1709,467610,成功,CHENGGONG,署屬有人氣象站,21.1,28.9,2018-12-04
1710,467620,蘭嶼,LANYU,署屬有人氣象站,20.6,25.5,2018-12-03
1711,467650,日月潭,SUN MOON LAKE,署屬有人氣象站,17.8,27.7,2018-12-05
1712,467660,臺東,TAITUNG,署屬有人氣象站,22.0,29.6,2018-12-04


In [4]:
import os
import pandas as pd

# Assuming all_dfs is your combined DataFrame

# Group by 'StationName'
grouped = all_dfs.groupby('StationName')

# Accessing each group
for name, group in grouped:
    print(f"Station: {name}")
    
    if name == '田中':
        print("資訊不足\n")
        continue
    
    
    station_name_folder = f'./{name}'
    if not os.path.isdir(station_name_folder):
        os.mkdir(station_name_folder)


    # Convert 'AirTemperatureMaxDate' to datetime
    group['AirTemperatureMaxDate'] = pd.to_datetime(group['AirTemperatureMaxDate'])

    # Create daily-like entries for the entire month
    df_daily = group.assign(AirTemperatureMaxDate=group['AirTemperatureMaxDate'].apply(
        lambda x: pd.date_range(start=x.replace(day=1), end=x.replace(day=1) + pd.offsets.MonthEnd(0), freq='D')
    )).explode('AirTemperatureMaxDate').reset_index(drop=True)

    # Sort date from earliest to latest
    df_daily = df_daily.sort_values(by='AirTemperatureMaxDate')

    # Drop duplicate-date rows
    df_daily = df_daily.drop_duplicates(subset=['AirTemperatureMaxDate'])
    
    # Select desired columns
    df_daily = df_daily.drop(columns=['StationID', 'StationNameEN', 'StationAttribute'])

    # Group df_daily by year
    grouped_by_year = df_daily.groupby(df_daily['AirTemperatureMaxDate'].dt.year)

    # Accessing each year within the station
    for year, df_year in grouped_by_year:
        print(f"  Year: {year}")

        # Display the result for each year
        display(df_year)
        
        # save it as csv
        df_year.to_csv(f'{station_name_folder}/{year}AirTemperature.csv', index=False)


    print("\n")


Station: 嘉義
  Year: 2018


,StationName,AirTemperatureMean,AirTemperatureMax,AirTemperatureMaxDate
1461,嘉義,17.4,27.8,2018-01-01
1462,嘉義,17.4,27.8,2018-01-02
1463,嘉義,17.4,27.8,2018-01-03
1464,嘉義,17.4,27.8,2018-01-04
1465,嘉義,17.4,27.8,2018-01-05
...,...,...,...,...
1821,嘉義,20.7,31.6,2018-12-27
1822,嘉義,20.7,31.6,2018-12-28
1823,嘉義,20.7,31.6,2018-12-29
1824,嘉義,20.7,31.6,2018-12-30


  Year: 2019


,StationName,AirTemperatureMean,AirTemperatureMax,AirTemperatureMaxDate
1096,嘉義,19.2,30.9,2019-01-01
1097,嘉義,19.2,30.9,2019-01-02
1098,嘉義,19.2,30.9,2019-01-03
1099,嘉義,19.2,30.9,2019-01-04
1100,嘉義,19.2,30.9,2019-01-05
...,...,...,...,...
1456,嘉義,19.1,30.0,2019-12-27
1457,嘉義,19.1,30.0,2019-12-28
1458,嘉義,19.1,30.0,2019-12-29
1459,嘉義,19.1,30.0,2019-12-30


  Year: 2020


...

,StationName,AirTemperatureMean,AirTemperatureMax,AirTemperatureMaxDate
761,高雄,20.9,29.0,2020-01-01
762,高雄,20.9,29.0,2020-01-02
763,高雄,20.9,29.0,2020-01-03
764,高雄,20.9,29.0,2020-01-04
765,高雄,20.9,29.0,2020-01-05
...,...,...,...,...
1122,高雄,22.3,28.6,2020-12-27
1123,高雄,22.3,28.6,2020-12-28
1124,高雄,22.3,28.6,2020-12-29
1125,高雄,22.3,28.6,2020-12-30


  Year: 2021


,StationName,AirTemperatureMean,AirTemperatureMax,AirTemperatureMaxDate
396,高雄,18.3,26.3,2021-01-01
397,高雄,18.3,26.3,2021-01-02
398,高雄,18.3,26.3,2021-01-03
399,高雄,18.3,26.3,2021-01-04
400,高雄,18.3,26.3,2021-01-05
...,...,...,...,...
756,高雄,21.2,28.1,2021-12-27
757,高雄,21.2,28.1,2021-12-28
758,高雄,21.2,28.1,2021-12-29
759,高雄,21.2,28.1,2021-12-30


  Year: 2022


,StationName,AirTemperatureMean,AirTemperatureMax,AirTemperatureMaxDate
31,高雄,19.1,27.0,2022-01-01
32,高雄,19.1,27.0,2022-01-02
2,高雄,20.1,27.7,2022-01-03
34,高雄,19.1,27.0,2022-01-04
35,高雄,19.1,27.0,2022-01-05
...,...,...,...,...
391,高雄,19.0,30.0,2022-12-27
392,高雄,19.0,30.0,2022-12-28
393,高雄,19.0,30.0,2022-12-29
394,高雄,19.0,30.0,2022-12-30


In [ ]:
directory = ''
folders = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]